In [1]:
#install libs

!pip install pandas scikit-learn nltk
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
ERROR: Will not install to the user site because it will lack sys.path precedence to pip in /usr/local/venv/lib/python3.12/site-packages


In [3]:

# Rule-Based Keyword Threshold Method

# import Libraries
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

nltk.download('stopwords')
from nltk.corpus import stopwords

# Load Dataset
df = pd.read_csv("cleaned_dataset.csv")
df = df[['Category', 'Resume']]

# Clean Text Function
def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]
    return ' '.join(words)

df['Cleaned_Resume'] = df['Resume'].apply(clean_text)

# Keyword List
keywords = pd.read_csv("keywords.csv")['Skill'].tolist()

def keyword_score(text):
    return sum([1 for kw in keywords if kw in text])

df['KeywordScore'] = df['Cleaned_Resume'].apply(keyword_score)

# Rule Based Threshold Labeling
df['Label'] = df['KeywordScore'].apply(lambda x: 1 if x >= 3 else 0)

# Model Training
X = df['Cleaned_Resume']
y = df['Label']

vectorizer = CountVectorizer(max_features=1000)
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=['Not a good fit', 'Fit']))


df['Prediction'] = model.predict(vectorizer.transform(df['Cleaned_Resume']))
df['Decision'] = df['Prediction'].apply(lambda x: "Fit – Move forward with interview" if x == 1 else "Not a good fit")

# Show Resume Decisions
#for i, row in df.iterrows():
  #  print(f"Resume #{i+1}: {row['Decision']} (KeywordScore: {row['KeywordScore']})")


[nltk_data] Downloading package stopwords to /home/np819/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                precision    recall  f1-score   support

Not a good fit       1.00      1.00      1.00        73
           Fit       1.00      1.00      1.00       120

      accuracy                           1.00       193
     macro avg       1.00      1.00      1.00       193
  weighted avg       1.00      1.00      1.00       193



In [4]:
# Export results to CSV
df[['Resume', 'Decision']].to_csv("results_rb.csv", index=False)